In [2]:
!pip install -U albumentations

     -------------------------------------- 224.0/224.0 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.16
    Uninstalling albumentations-1.4.16:
      Successfully uninstalled albumentations-1.4.16


In [2]:
!pip install --upgrade paramiko cryptography


     -------------------------------------- 227.1/227.1 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 3.1/3.1 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 37.0.1
    Uninstalling cryptography-37.0.1:
      Successfully uninstalled cryptography-37.0.1
  Attempting uninstall: paramiko
    Found existing installation: paramiko 2.8.1
    Uninstalling paramiko-2.8.1:
      Successfully uninstalled paramiko-2.8.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.


In [1]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import albumentations as A

In [2]:
#Path
image_dir = 'Data_Set_Larch_Casebearer/Imagedata'
annotation_dir  = 'Data_Set_Larch_Casebearer/Annotations'
save_dir = 'Data_Set_Larch_Casebearer/Croppedimages'
output_annotation_dir = 'Data_Set_Larch_Casebearer/Augmented_Annotations'


In [3]:
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bboxes = []
    labels = []

   
    for obj in root.findall('object'):
        # Check if the 'damage' tag exists
        damage_tag = obj.find('damage')
        if damage_tag is not None:
            label = damage_tag.text  # Healthy, Light Damage, High Damage, Others
        else:
            label = 'unknown'  # Handle the case where 'damage' tag is missing

        # Parse bounding box coordinates
        bndbox = obj.find('bndbox')
        if bndbox is not None:
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            bboxes.append([xmin, ymin, xmax, ymax])
            labels.append(label)

    return bboxes, labels

In [6]:
import os
import cv2

# Function to crop the images based on bounding boxes and save them
def crop_and_save_images(image_dir, annotation_dir, save_dir):
    # Iterate over all images and corresponding annotations
    for img_file in os.listdir(image_dir):
        if img_file.endswith('.JPG') or img_file.endswith('.png'):
            img_path = os.path.join(image_dir, img_file)
            
            # Corresponding XML annotation file
            xml_file = os.path.join(annotation_dir, os.path.splitext(img_file)[0] + '.xml')
            
            # Check if the XML file exists
            if os.path.exists(xml_file):
                # Read image
                img = cv2.imread(img_path)
                
                # Check if the image is loaded correctly
                if img is None:
                    print(f"Error: Unable to load image {img_path}")
                    continue  # Skip this image and continue with the next one

                # Parse the XML file to get bounding boxes and labels
                bboxes, labels = parse_xml(xml_file)

                height, width, _ = img.shape
                
                # Crop and save the trees
                for i, bbox in enumerate(bboxes):
                    xmin, ymin, xmax, ymax = bbox

                    # Validate bounding boxes (make sure they are within the image boundaries)
                    if xmin < 0 or ymin < 0 or xmax > width or ymax > height:
                        print(f"Warning: Bounding box {bbox} is out of bounds for image {img_file}")
                        continue  # Skip this bounding box

                    # Crop the region defined by the bounding box
                    cropped_img = img[ymin:ymax, xmin:xmax]

                    # Ensure the cropped image is not empty
                    if cropped_img.size > 0:
                        label = labels[i]

                        # Ensure save directory exists
                        if not os.path.exists(save_dir):
                            os.makedirs(save_dir)

                        # Save cropped image based on label
                        cropped_save_path = os.path.join(save_dir, f'{label}_{os.path.splitext(img_file)[0]}_{i}.jpg')
                        cv2.imwrite(cropped_save_path, cropped_img)
                    else:
                        print(f"Error: Cropped image is empty for bounding box {bbox} in {img_file}")
            else:
                print(f"Error: Annotation file not found for {img_file}")
                continue


In [7]:
crop_and_save_images(image_dir, annotation_dir , save_dir)

Error: Annotation file not found for B01_0048.JPG
Error: Cropped image is empty for bounding box [430, 1084, 430, 1086] in B05_0073.JPG
Error: Cropped image is empty for bounding box [603, 1330, 603, 1331] in B08_0008.JPG
Error: Cropped image is empty for bounding box [473, 864, 473, 866] in B09_0010.JPG


In [17]:
bbox_params = A.BboxParams(format='pascal_voc', label_fields=['class_labels'])

In [18]:
#augmentation pipeline
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),                # Random horizontal flip
    A.VerticalFlip(p=0.5),                  # Random vertical flip
    A.RandomRotate90(p=0.5),                 # Random 90-degree rotation      # Random brightness and contrast adjustments
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),  # Random shift, scale, and rotation
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),  # Normalize the image
], bbox_params=bbox_params, additional_targets={'mask': 'mask'})  # Ensure the same transformations apply to the mask

In [19]:
# Function to load annotations from XML
def load_annotations(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = []
    for obj in root.findall('object'):
        class_name = obj.find('damage').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        annotations.append((class_name, (xmin, ymin, xmax, ymax)))
    return annotations

In [20]:
def save_annotations(xml_file, annotations):
    root = ET.Element("annotation")
    for class_name, (xmin, ymin, xmax, ymax) in annotations:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "damage").text = class_name
        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(xmin)
        ET.SubElement(bndbox, "ymin").text = str(ymin)
        ET.SubElement(bndbox, "xmax").text = str(xmax)
        ET.SubElement(bndbox, "ymax").text = str(ymax)
    
    tree = ET.ElementTree(root)
    tree.write(xml_file)

In [21]:
# Loop through all images and their corresponding annotations
for image_file in os.listdir(image_dir):
    if image_file.endswith('.JPG') or image_file.endswith('.png'):
        image_path = os.path.join(image_dir, image_file)
        xml_file = os.path.join(annotation_dir, image_file.replace('.JPG', '.xml').replace('.png', '.xml'))
        
        # Check if the XML file exists
        if not os.path.exists(xml_file):
            print(f"Warning: Annotation file {xml_file} not found. Skipping {image_file}.")
            continue
        
        # Load the image
        image = cv2.imread(image_path)
        
        # Load annotations
        annotations = load_annotations(xml_file)
        
        # Perform augmentation
        augmented = augmentation(image=image)
        augmented_image = augmented['image']
        
        # Save augmented image
        augmented_image_file = os.path.join(output_image_dir, image_file)
        cv2.imwrite(augmented_image_file, augmented_image)
        
        # Update bounding box coordinates based on augmentation
        updated_annotations = []
        for class_name, (xmin, ymin, xmax, ymax) in annotations:
            # Apply the inverse transformations to the bounding boxes
            transformed_bbox = augmentation(image=image, bboxes=[(xmin, ymin, xmax, ymax)])['bboxes'][0]
            updated_annotations.append((class_name, transformed_bbox))

        # Save updated annotations in a new XML file
        new_xml_file = os.path.join(output_annotation_dir, image_file.replace('.JPG', '.xml').replace('.png', '.xml'))
        save_annotations(new_xml_file, updated_annotations)

print("Augmentation complete!")

ValueError: Your 'label_fields' are not valid - them must have same names as params in dict